# Pre-Equilibrium Maximazation of Yield #

In [1]:
# make sure jupyter path is correct for loading local moudules
import sys
# path to steric_simulator module relative to notebook
sys.path.append("../../../")
import copy

In [2]:
from steric_free_simulator import ReactionNetwork, VectorizedRxnNet, VecSim, Optimizer, EquilibriumSolver
import networkx as nx
import torch
from torch import DoubleTensor as Tensor
import numpy as np

EnergyExplorer Module is not available. Check Rosetta installation. <ipykernel.iostream.OutStream object at 0x155550ac4128>


We'll start with the AP2 complex that we've worked with before. Pairwise $\Delta Gs$ were derived from the PDB structures via Rossetta

In [3]:
base_input = '../../input_files/ap2_coup.pwr'
rn = ReactionNetwork(base_input, one_step=True)
rn.resolve_tree()
vec_rn = VectorizedRxnNet(rn, dev='cpu')

['default_assoc', 1.0]
['rxn_coupling', True]
True
['monomer_add_only', -1]
['A']
100.0
['M']
100.0
['B']
100.0
['S']
100.0
Parsing rule...
SPLIT_01:  ['A(a)+B(b)', 'A(a!1).B(a!1)']
['A', 'B', '']
GGGGGGGGGgg
Parsing rule...
SPLIT_01:  ['A(b)+M(a)', 'A(b!1).M(a!1)']
['A', 'M', '']
GGGGGGGGGgg
Parsing rule...
SPLIT_01:  ['A(c)+S(a)', 'A(c!1).S(a!1)']
['A', 'S', '']
GGGGGGGGGgg
Parsing rule...
SPLIT_01:  ['B(b)+M(b)', 'B(b!1).M(b!1)']
['B', 'M', '']
GGGGGGGGGgg
Parsing rule...
SPLIT_01:  ['M(c)+S(b)', 'M(c!1).S(b!1)']
['M', 'S', '']
GGGGGGGGGgg
Parsing rule...
SPLIT_01:  ['B(c)+S(b)', 'M(c!1).S(b!1)']
['B', 'S', '']
GGGGGGGGGgg
Node-1 :  (0, {'struct': <networkx.classes.graph.Graph object at 0x1554bb820828>, 'copies': tensor([100.], dtype=torch.float64), 'subunits': 1})
Node-2 :  (0, {'struct': <networkx.classes.graph.Graph object at 0x1554bb820828>, 'copies': tensor([100.], dtype=torch.float64), 'subunits': 1})
-----
{'A'}
{'A'}
set()
Steric hindrance detected
Node-1 :  (0, {'struct': <

Node-1 :  (8, {'struct': <networkx.classes.graph.Graph object at 0x1554bb4cd160>, 'copies': tensor([0.], dtype=torch.float64), 'subunits': 2})
Node-2 :  (9, {'struct': <networkx.classes.graph.Graph object at 0x1554bb536128>, 'copies': tensor([0.], dtype=torch.float64), 'subunits': 6})
-----
{'S', 'M'}
{'B', 'A', 'M'}
{'S'}
Steric hindrance detected
Node-1 :  (8, {'struct': <networkx.classes.graph.Graph object at 0x1554bb4cd160>, 'copies': tensor([0.], dtype=torch.float64), 'subunits': 2})
Node-2 :  (10, {'struct': <networkx.classes.graph.Graph object at 0x1554bb4cd7f0>, 'copies': tensor([0.], dtype=torch.float64), 'subunits': 6})
-----
{'S', 'M'}
{'S', 'A', 'M'}
set()
Steric hindrance detected
Node-1 :  (8, {'struct': <networkx.classes.graph.Graph object at 0x1554bb4cd160>, 'copies': tensor([0.], dtype=torch.float64), 'subunits': 2})
Node-2 :  (11, {'struct': <networkx.classes.graph.Graph object at 0x1554bb4cd278>, 'copies': tensor([0.], dtype=torch.float64), 'subunits': 2})
-----
{'S'

In [4]:
uid_dict = {}
sys.path.append("../../")
import numpy as np
from reaction_network import gtostr
for n in rn.network.nodes():
    #print(n)
    #print(rn.network.nodes()[n])
    print(n,"--",gtostr(rn.network.nodes[n]['struct']))
    for k,v in rn.network[n].items():
        uid = v['uid']
        r1 = set(gtostr(rn.network.nodes[n]['struct']))
        p = set(gtostr(rn.network.nodes[k]['struct']))
        r2 = p-r1
        reactants = (r1,r2)
        uid_dict[(n,k)] = uid

print(uid_dict)

0 -- A
1 -- M
2 -- B
3 -- S
4 -- AM
5 -- AB
6 -- AS
7 -- BM
8 -- MS
9 -- ABM
10 -- AMS
11 -- BS
12 -- ABS
13 -- BMS
14 -- ABMS
{(0, 4): 0, (0, 5): 1, (0, 6): 2, (0, 9): 19, (0, 10): 20, (0, 12): 21, (0, 14): 24, (1, 4): 0, (1, 7): 3, (1, 8): 4, (1, 9): 5, (1, 10): 6, (1, 13): 22, (1, 14): 23, (2, 5): 1, (2, 7): 3, (2, 11): 7, (2, 9): 8, (2, 12): 9, (2, 13): 10, (2, 14): 11, (3, 6): 2, (3, 8): 4, (3, 11): 7, (3, 10): 12, (3, 12): 13, (3, 13): 14, (3, 14): 15, (4, 9): 8, (4, 10): 12, (4, 14): 16, (5, 9): 5, (5, 12): 13, (5, 14): 17, (6, 10): 6, (6, 12): 9, (6, 14): 18, (7, 13): 14, (7, 14): 18, (7, 9): 19, (8, 13): 10, (8, 14): 17, (8, 10): 20, (9, 14): 15, (10, 14): 11, (11, 14): 16, (11, 12): 21, (11, 13): 22, (12, 14): 23, (13, 14): 24}


In [5]:
#Do modifications here
#Changing Initial Conditions
import networkx as nx
#Changin k_on
new_kon = torch.zeros([rn._rxn_count], requires_grad=True).double()
new_kon = new_kon + Tensor([1.]*np.array(1e-1))
new_kon[0:3]=3
# new_kon[1]=5
# new_kon[2]=0.5
update_kon_dict = {}
for edge in rn.network.edges:
    print(rn.network.get_edge_data(edge[0],edge[1]))
    update_kon_dict[edge] = new_kon[uid_dict[edge]]

nx.set_edge_attributes(rn.network,update_kon_dict,'k_on')

# for edge in rn.network.edges:
#     print(rn.network.get_edge_data(edge[0],edge[1]))

optim_rates=[0,1,2]
# slow_rates=[2]
# vec_rn = VectorizedRxnNet(rn, dev='cpu',optim_rates=optim_rates,slow_rates=slow_rates,slow_ratio=10)
vec_rn = VectorizedRxnNet(rn, dev='cpu',optim_rates=optim_rates)
print(vec_rn.kon)

#Changing initial concentrations

{'k_on': 1.0, 'k_off': None, 'lcf': 1, 'rxn_score': tensor([-100.], dtype=torch.float64), 'uid': 0}
{'k_on': 1.0, 'k_off': None, 'lcf': 1, 'rxn_score': tensor([-100.], dtype=torch.float64), 'uid': 1}
{'k_on': 1.0, 'k_off': None, 'lcf': 1, 'rxn_score': tensor([-100.], dtype=torch.float64), 'uid': 2}
{'k_on': 1.0, 'k_off': None, 'lcf': 1, 'rxn_score': tensor([-200.], dtype=torch.float64), 'uid': 19}
{'k_on': 1.0, 'k_off': None, 'lcf': 1, 'rxn_score': tensor([-200.], dtype=torch.float64), 'uid': 20}
{'k_on': 1.0, 'k_off': None, 'lcf': 1, 'rxn_score': tensor([-200.], dtype=torch.float64), 'uid': 21}
{'k_on': 1.0, 'k_off': None, 'lcf': 1, 'rxn_score': tensor([-300.], dtype=torch.float64), 'uid': 24}
{'k_on': 1.0, 'k_off': None, 'lcf': 1, 'rxn_score': tensor([-100.], dtype=torch.float64), 'uid': 0}
{'k_on': 1.0, 'k_off': None, 'lcf': 1, 'rxn_score': tensor([-100.], dtype=torch.float64), 'uid': 3}
{'k_on': 1.0, 'k_off': None, 'lcf': 1, 'rxn_score': tensor([-100.], dtype=torch.float64), 'uid':

In [6]:
print(vec_rn.rxn_class)

{(1, 1): [0, 1, 2, 3, 4, 7], (2, 1): [5, 6, 8, 9, 10, 12, 13, 14, 19, 20, 21, 22], (3, 1): [11, 15, 23, 24], (2, 2): [16, 17, 18]}


## The Equilibrium Solution ##
First we will find the equilibrium solution for this system.

In [7]:
# vec_rn.update_reaction_net(rn)
# poly_system = EquilibriumSolver(rn)
# solution = poly_system.solve()
# print(solution)
# if solution == None:
#     print("No Equilibrium solution")
# else:
#     print(solution)
#     print("Equilibrium expected yield: ", 100 * solution[-1] / min(vec_rn.initial_copies[:vec_rn.num_monomers]), '%')
# print(vec_rn.kon)

In [8]:
uid_dict = {}
sys.path.append("../")
import numpy as np
from reaction_network import gtostr
from torch import DoubleTensor as Tensor

def get_max_edge(n):
    """
    Calculates the max rate (k_on) for a given node
    To find out the maximum flow path to the final complex starting from the current node.
    
    Can also calculate the total rate of consumption of a node by summing up all rates. 
    Can tell which component is used quickly.
    """
    try:
        edges = rn.network.out_edges(n)
        #Loop over all edges
        #Get attributes
        if len(edges)==0:
            return(False)
        kon_max = -1
        next_node = -1
        
        kon_sum = 0
        for edge in edges:
            data = rn.network.get_edge_data(edge[0],edge[1])
            #print(data)
            #Get uid
            uid = data['uid']
            #Get updated kon
            temp_kon = vec_rn.kon[uid]
            kon_sum+=temp_kon
            
#             #Calculate k_off also
#             std_c = Tensor([1.])
#             l_kon = torch.log(temp_kon)
#             l_koff = (vec_rn.rxn_score_vec[uid] * 1. / (self._R * self._T)) + l_kon + torch.log(std_c)
            if temp_kon > kon_max:
                kon_max = temp_kon
                next_node=edge[1]
        return(kon_max,next_node,kon_sum)
    except Exception as err:
        raise(err)

pathway = []
kon_sumarray = []
total_con_rate = {}
for n in rn.network.nodes():
    
    n_str = gtostr(rn.network.nodes[n]['struct']) 
    
    paths = [n_str]
    kon_sum = 0
    temp_node = n
    max_edge = True
    consumption_rate = 0
    if n < len(rn.network.nodes()):#num_monomers:
#         print("Current node: ")
#         print(n_str)
        while max_edge:
            max_edge = get_max_edge(temp_node)
            if max_edge:
                total_con_rate[gtostr(rn.network.nodes[temp_node]['struct'])] = max_edge[2]
                temp_node = max_edge[1]
                kon_sum += max_edge[0].item()
                
#                 print("Next node: ")
#                 print(temp_node)

                paths.append(gtostr(rn.network.nodes[temp_node]['struct']))
            else:
                break
        pathway.append(paths)
        kon_sumarray.append(kon_sum)
        paths=[]

print(pathway)
print(kon_sumarray)
#print(total_con_rate)

[['A', 'AM', 'ABM', 'ABMS'], ['M', 'AM', 'ABM', 'ABMS'], ['B', 'AB', 'ABM', 'ABMS'], ['S', 'AS', 'AMS', 'ABMS'], ['AM', 'ABM', 'ABMS'], ['AB', 'ABM', 'ABMS'], ['AS', 'AMS', 'ABMS'], ['BM', 'BMS', 'ABMS'], ['MS', 'BMS', 'ABMS'], ['ABM', 'ABMS'], ['AMS', 'ABMS'], ['BS', 'ABMS'], ['ABS', 'ABMS'], ['BMS', 'ABMS'], ['ABMS']]
[3.2, 3.2, 3.2, 3.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.1, 0.1, 0.1, 0.1, 0.1, 0]


In [9]:
# if solution != None:
#     for k,v in sorted(total_con_rate.items(),key=lambda x : x[1]):
#         print(k," : ", v.item())

In [10]:
# if solution !=None:
#     for k,v in sorted(net_flux.items(),key=lambda x : x[1]):
#         print(k," : ", v)

## Using the optimizer with a 1 second simulation runtime ##

In [11]:
vec_rn.reset(reset_params=True)
optim = Optimizer(reaction_network=vec_rn,
                  sim_runtime=1,
                  optim_iterations=1000,
                  learning_rate=1e-2,
                  device='cpu',method="RMSprop",)
optim.rn.update_reaction_net(rn)
optim.optimize(conc_scale=1,conc_thresh=1,max_thresh=10,yield_species=14,mod_bool=True)

Using CPU
Reaction Parameters before optimization: 
[Parameter containing:
tensor([3., 3., 3.], dtype=torch.float64, requires_grad=True)]
Optimizer State: <bound method Optimizer.state_dict of RMSprop (
Parameter Group 0
    alpha: 0.99
    centered: False
    eps: 1e-08
    lr: 0.01
    momentum: 0
    weight_decay: 0
)>
Using CPU
Start of simulation: memory Used:  19.3
SLow rates:  tensor([[3.0000, 3.0000, 3.0000, 0.1000, 0.1000, 3.0000, 3.0000, 0.1000, 3.0000,
         3.0000, 0.1000, 3.0000, 3.0000, 3.0000, 0.1000, 3.0000, 3.0000, 3.0000,
         3.0000, 3.0000, 3.0000, 3.0000, 0.1000, 3.0000, 3.0000]],
       dtype=torch.float64, grad_fn=<UnsqueezeBackward0>)
Next time:  tensor(1.0507, dtype=torch.float64, grad_fn=<AddBackward0>)
yield on sim iteration 0 was 89.9%
current params: tensor([3., 3., 3.], dtype=torch.float64)
Using CPU
Start of simulation: memory Used:  19.3
SLow rates:  tensor([[3.1000, 3.1000, 3.1000, 0.1000, 0.1000, 3.1000, 3.1000, 0.1000, 3.1000,
         3.1000, 

Using CPU
Start of simulation: memory Used:  19.4
SLow rates:  tensor([[3.4321, 3.6119, 3.7056, 0.1000, 0.1000, 3.4321, 3.4321, 0.1000, 3.6119,
         3.6119, 0.1000, 3.6119, 3.7056, 3.7056, 0.1000, 3.7056, 3.7056, 3.7056,
         3.6119, 3.6119, 3.7056, 3.7056, 0.1000, 3.4321, 3.7056]],
       dtype=torch.float64, grad_fn=<UnsqueezeBackward0>)
Next time:  tensor(1.0112, dtype=torch.float64, grad_fn=<AddBackward0>)
yield on sim iteration 15 was 91.3%
current params: tensor([3.4321, 3.6119, 3.7056], dtype=torch.float64)
Using CPU
Start of simulation: memory Used:  19.4
SLow rates:  tensor([[3.4490, 3.6350, 3.7302, 0.1000, 0.1000, 3.4490, 3.4490, 0.1000, 3.6350,
         3.6350, 0.1000, 3.6350, 3.7302, 3.7302, 0.1000, 3.7302, 3.7302, 3.7302,
         3.6350, 3.6350, 3.7302, 3.7302, 0.1000, 3.4490, 3.7302]],
       dtype=torch.float64, grad_fn=<UnsqueezeBackward0>)
Next time:  tensor(1.0076, dtype=torch.float64, grad_fn=<AddBackward0>)
yield on sim iteration 16 was 91.4%
current params

Next time:  tensor(1.0351, dtype=torch.float64, grad_fn=<AddBackward0>)
yield on sim iteration 30 was 91.9%
current params: tensor([3.6593, 3.9045, 4.0115], dtype=torch.float64)
Using CPU
Start of simulation: memory Used:  19.4
SLow rates:  tensor([[3.6729, 3.9211, 4.0285, 0.1000, 0.1000, 3.6729, 3.6729, 0.1000, 3.9211,
         3.9211, 0.1000, 3.9211, 4.0285, 4.0285, 0.1000, 4.0285, 4.0285, 4.0285,
         3.9211, 3.9211, 4.0285, 4.0285, 0.1000, 3.6729, 4.0285]],
       dtype=torch.float64, grad_fn=<UnsqueezeBackward0>)
Next time:  tensor(1.0330, dtype=torch.float64, grad_fn=<AddBackward0>)
yield on sim iteration 31 was 91.9%
current params: tensor([3.6729, 3.9211, 4.0285], dtype=torch.float64)
Using CPU
Start of simulation: memory Used:  19.4
SLow rates:  tensor([[3.6863, 3.9374, 4.0453, 0.1000, 0.1000, 3.6863, 3.6863, 0.1000, 3.9374,
         3.9374, 0.1000, 3.9374, 4.0453, 4.0453, 0.1000, 4.0453, 4.0453, 4.0453,
         3.9374, 3.9374, 4.0453, 4.0453, 0.1000, 3.6863, 4.0453]],
  

Next time:  tensor(1.0072, dtype=torch.float64, grad_fn=<AddBackward0>)
yield on sim iteration 46 was 92.3%
current params: tensor([3.8618, 4.1444, 4.2574], dtype=torch.float64)
Using CPU
Start of simulation: memory Used:  19.5
SLow rates:  tensor([[3.8735, 4.1580, 4.2713, 0.1000, 0.1000, 3.8735, 3.8735, 0.1000, 4.1580,
         4.1580, 0.1000, 4.1580, 4.2713, 4.2713, 0.1000, 4.2713, 4.2713, 4.2713,
         4.1580, 4.1580, 4.2713, 4.2713, 0.1000, 3.8735, 4.2713]],
       dtype=torch.float64, grad_fn=<UnsqueezeBackward0>)
Next time:  tensor(1.0058, dtype=torch.float64, grad_fn=<AddBackward0>)
yield on sim iteration 47 was 92.3%
current params: tensor([3.8735, 4.1580, 4.2713], dtype=torch.float64)
Using CPU
Start of simulation: memory Used:  19.5
SLow rates:  tensor([[3.8852, 4.1715, 4.2850, 0.1000, 0.1000, 3.8852, 3.8852, 0.1000, 4.1715,
         4.1715, 0.1000, 4.1715, 4.2850, 4.2850, 0.1000, 4.2850, 4.2850, 4.2850,
         4.1715, 4.1715, 4.2850, 4.2850, 0.1000, 3.8852, 4.2850]],
  

Next time:  tensor(1.0492, dtype=torch.float64, grad_fn=<AddBackward0>)
yield on sim iteration 62 was 92.6%
current params: tensor([4.0411, 4.3481, 4.4646], dtype=torch.float64)
Using CPU
Start of simulation: memory Used:  19.6
SLow rates:  tensor([[4.0517, 4.3600, 4.4767, 0.1000, 0.1000, 4.0517, 4.0517, 0.1000, 4.3600,
         4.3600, 0.1000, 4.3600, 4.4767, 4.4767, 0.1000, 4.4767, 4.4767, 4.4767,
         4.3600, 4.3600, 4.4767, 4.4767, 0.1000, 4.0517, 4.4767]],
       dtype=torch.float64, grad_fn=<UnsqueezeBackward0>)
Next time:  tensor(1.0481, dtype=torch.float64, grad_fn=<AddBackward0>)
yield on sim iteration 63 was 92.6%
current params: tensor([4.0517, 4.3600, 4.4767], dtype=torch.float64)
Using CPU
Start of simulation: memory Used:  19.6
SLow rates:  tensor([[4.0623, 4.3718, 4.4887, 0.1000, 0.1000, 4.0623, 4.0623, 0.1000, 4.3718,
         4.3718, 0.1000, 4.3718, 4.4887, 4.4887, 0.1000, 4.4887, 4.4887, 4.4887,
         4.3718, 4.3718, 4.4887, 4.4887, 0.1000, 4.0623, 4.4887]],
  

...

Next time:  tensor(1.0274, dtype=torch.float64, grad_fn=<AddBackward0>)
yield on sim iteration 929 was 96.9%
current params: tensor([11.0551, 11.4649, 11.6053], dtype=torch.float64)
Using CPU
Start of simulation: memory Used:  23.8
SLow rates:  tensor([[11.0635, 11.4733, 11.6137,  0.1000,  0.1000, 11.0635, 11.0635,  0.1000,
         11.4733, 11.4733,  0.1000, 11.4733, 11.6137, 11.6137,  0.1000, 11.6137,
         11.6137, 11.6137, 11.4733, 11.4733, 11.6137, 11.6137,  0.1000, 11.0635,
         11.6137]], dtype=torch.float64, grad_fn=<UnsqueezeBackward0>)
Next time:  tensor(1.0272, dtype=torch.float64, grad_fn=<AddBackward0>)
yield on sim iteration 930 was 96.9%
current params: tensor([11.0635, 11.4733, 11.6137], dtype=torch.float64)
Using CPU
Start of simulation: memory Used:  23.8
SLow rates:  tensor([[11.0718, 11.4817, 11.6221,  0.1000,  0.1000, 11.0718, 11.0718,  0.1000,
         11.4817, 11.4817,  0.1000, 11.4817, 11.6221, 11.6221,  0.1000, 11.6221,
         11.6221, 11.6221, 11.4817

Next time:  tensor(1.0247, dtype=torch.float64, grad_fn=<AddBackward0>)
yield on sim iteration 944 was 96.9%
current params: tensor([11.1809, 11.5907, 11.7312], dtype=torch.float64)
Using CPU
Start of simulation: memory Used:  23.9
SLow rates:  tensor([[11.1893, 11.5991, 11.7396,  0.1000,  0.1000, 11.1893, 11.1893,  0.1000,
         11.5991, 11.5991,  0.1000, 11.5991, 11.7396, 11.7396,  0.1000, 11.7396,
         11.7396, 11.7396, 11.5991, 11.5991, 11.7396, 11.7396,  0.1000, 11.1893,
         11.7396]], dtype=torch.float64, grad_fn=<UnsqueezeBackward0>)
Next time:  tensor(1.0245, dtype=torch.float64, grad_fn=<AddBackward0>)
yield on sim iteration 945 was 96.9%
current params: tensor([11.1893, 11.5991, 11.7396], dtype=torch.float64)
Using CPU
Start of simulation: memory Used:  23.9
SLow rates:  tensor([[11.1977, 11.6074, 11.7480,  0.1000,  0.1000, 11.1977, 11.1977,  0.1000,
         11.6074, 11.6074,  0.1000, 11.6074, 11.7480, 11.7480,  0.1000, 11.7480,
         11.7480, 11.7480, 11.6074

Next time:  tensor(1.0220, dtype=torch.float64, grad_fn=<AddBackward0>)
yield on sim iteration 959 was 96.9%
current params: tensor([11.3069, 11.7166, 11.8573], dtype=torch.float64)
Using CPU
Start of simulation: memory Used:  24.0
SLow rates:  tensor([[11.3153, 11.7250, 11.8657,  0.1000,  0.1000, 11.3153, 11.3153,  0.1000,
         11.7250, 11.7250,  0.1000, 11.7250, 11.8657, 11.8657,  0.1000, 11.8657,
         11.8657, 11.8657, 11.7250, 11.7250, 11.8657, 11.8657,  0.1000, 11.3153,
         11.8657]], dtype=torch.float64, grad_fn=<UnsqueezeBackward0>)
Next time:  tensor(1.0218, dtype=torch.float64, grad_fn=<AddBackward0>)
yield on sim iteration 960 was 96.9%
current params: tensor([11.3153, 11.7250, 11.8657], dtype=torch.float64)
Using CPU
Start of simulation: memory Used:  24.0
SLow rates:  tensor([[11.3238, 11.7335, 11.8741,  0.1000,  0.1000, 11.3238, 11.3238,  0.1000,
         11.7335, 11.7335,  0.1000, 11.7335, 11.8741, 11.8741,  0.1000, 11.8741,
         11.8741, 11.8741, 11.7335

Next time:  tensor(1.0194, dtype=torch.float64, grad_fn=<AddBackward0>)
yield on sim iteration 974 was 96.9%
current params: tensor([11.4332, 11.8429, 11.9836], dtype=torch.float64)
Using CPU
Start of simulation: memory Used:  24.1
SLow rates:  tensor([[11.4416, 11.8513, 11.9921,  0.1000,  0.1000, 11.4416, 11.4416,  0.1000,
         11.8513, 11.8513,  0.1000, 11.8513, 11.9921, 11.9921,  0.1000, 11.9921,
         11.9921, 11.9921, 11.8513, 11.8513, 11.9921, 11.9921,  0.1000, 11.4416,
         11.9921]], dtype=torch.float64, grad_fn=<UnsqueezeBackward0>)
Next time:  tensor(1.0192, dtype=torch.float64, grad_fn=<AddBackward0>)
yield on sim iteration 975 was 96.9%
current params: tensor([11.4416, 11.8513, 11.9921], dtype=torch.float64)
Using CPU
Start of simulation: memory Used:  24.1
SLow rates:  tensor([[11.4501, 11.8597, 12.0005,  0.1000,  0.1000, 11.4501, 11.4501,  0.1000,
         11.8597, 11.8597,  0.1000, 11.8597, 12.0005, 12.0005,  0.1000, 12.0005,
         12.0005, 12.0005, 11.8597

Next time:  tensor(1.0168, dtype=torch.float64, grad_fn=<AddBackward0>)
yield on sim iteration 989 was 97.0%
current params: tensor([11.5597, 11.9693, 12.1102], dtype=torch.float64)
Using CPU
Start of simulation: memory Used:  24.1
SLow rates:  tensor([[11.5682, 11.9777, 12.1187,  0.1000,  0.1000, 11.5682, 11.5682,  0.1000,
         11.9777, 11.9777,  0.1000, 11.9777, 12.1187, 12.1187,  0.1000, 12.1187,
         12.1187, 12.1187, 11.9777, 11.9777, 12.1187, 12.1187,  0.1000, 11.5682,
         12.1187]], dtype=torch.float64, grad_fn=<UnsqueezeBackward0>)
Next time:  tensor(1.0166, dtype=torch.float64, grad_fn=<AddBackward0>)
yield on sim iteration 990 was 97.0%
current params: tensor([11.5682, 11.9777, 12.1187], dtype=torch.float64)
Using CPU
Start of simulation: memory Used:  24.1
SLow rates:  tensor([[11.5766, 11.9862, 12.1271,  0.1000,  0.1000, 11.5766, 11.5766,  0.1000,
         11.9862, 11.9862,  0.1000, 11.9862, 12.1271, 12.1271,  0.1000, 12.1271,
         12.1271, 12.1271, 11.9862

In [12]:
print(vec_rn.slow_rates)

None


In [13]:
# optim.optimize()

In [14]:
# optim.optimize()

In [15]:
def calc_var(v1,v2):
    sq_sum=0
    for i in range(len(v1)):
        sq_sum=(v1[i]-v2[i])**2+sq_sum
    
    sq_sum = ((sq_sum)**0.5)/(len(v1)-1)
    return(sq_sum)

In [16]:
yields= []
final_params=[]
for i in range(len(optim.final_yields)):
    yields.append(optim.final_yields[i].item())
#     print(optim.final_solns[i].numpy())
    final_params.append(optim.final_solns[i].numpy())

sort_indx=np.argsort(np.array(yields))
sorted_yields=np.array(yields)[sort_indx]
sorted_params = np.array(final_params)[sort_indx]

p0 = sorted_params[0]
var_params = []
for i in range(len(sorted_params)):
    var_params.append(calc_var(p0,sorted_params[i]))
    
arg_indx = np.argsort(np.array(var_params))
sorted_var = np.array(var_params)[arg_indx]

print(sorted_var[0])
print(sorted_var[-1])
print("Yield: ",sorted_yields[arg_indx[0]],"\nParams: ",sorted_params[arg_indx[0]])

print("Yield: ",sorted_yields[arg_indx[-1]],"\nParams: ",sorted_params[arg_indx[-1]])
print("Max Yield: ",sorted_yields[-1],"\nParams: ",sorted_params[-1])

0.0
7.75856631751148
Yield:  0.8993929203648626 
Params:  [3. 3. 3.]
Yield:  0.9704421326998083 
Params:  [11.63576887 12.04529028 12.18627373]
Max Yield:  0.9704421326998083 
Params:  [11.63576887 12.04529028 12.18627373]


In [17]:
# optim.optimize()


In [18]:
print(vec_rn.rxn_class)

uid_dict = {}
sys.path.append("../")

final_rxn_class = {}
import numpy as np
from reaction_network import gtostr
for n in rn.network.nodes():
    #print(n)
    #print(rn.network.nodes()[n])
    for k,v in rn.network[n].items():
        uid = v['uid']
        r1 = set(gtostr(rn.network.nodes[n]['struct']))
        p = set(gtostr(rn.network.nodes[k]['struct']))
        r2 = p-r1
        reactants = (r1,r2)
    
        #Find no. of bonds formed for this uid
        for cls,r_id in vec_rn.rxn_class.items():
            if uid in r_id:
                nb = cls

        #Formula to get separate id for each type of rxn
        key_id = abs(len(r1) - len(r2)) * nb + len(p)
#         print(reactants,key_id)
        
        if key_id not in final_rxn_class:
            final_rxn_class[key_id] = [uid]
        else:
            if uid not in final_rxn_class[key_id]:
                final_rxn_class[key_id].append(uid)
                
print(final_rxn_class)

#Code for labels
#Only for full topology

lb_rxn_class = {2:'mono-mono',5:'mono-dim',10:'mono-tri',4:'dim-dim'}

{(1, 1): [0, 1, 2, 3, 4, 7], (2, 1): [5, 6, 8, 9, 10, 12, 13, 14, 19, 20, 21, 22], (3, 1): [11, 15, 23, 24], (2, 2): [16, 17, 18]}


TypeError: can only concatenate tuple (not "int") to tuple

In [19]:
def calc_var(v1,v2):
    sq_sum=0
    for i in range(len(v1)):
        sq_sum=(v1[i]-v2[i])**2+sq_sum
    
    sq_sum = ((sq_sum)**0.5)/(len(v1)-1)
    return(sq_sum)

def calc_asymm(par):
    
    avg_rates = []
    var_rates = []
    rat1 = []
    rat2 =  []
    
    lb_1 = []
    lb_2 = []
    
    for rclass,uid in final_rxn_class.items():
        a1 = np.mean(par[uid])
        avg_rates.append(a1)
        lb_1.append("Avg rates - {:s}".format(lb_rxn_class[rclass]))
        
        var1 = np.var(par[uid])
        var_rates.append(var1)
        lb_2.append("Var rates - {:s}".format(lb_rxn_class[rclass]))
        
    final_val = avg_rates+var_rates
    final_lb = lb_1+lb_2
    return(final_val,final_lb)

In [20]:
print(vec_rn.rx_cid)
print(vec_rn.coup_map)

{5: [0, 3], 6: [0, 4], 8: [1, 3], 9: [1, 7], 10: [3, 7], 11: [1, 3, 7], 12: [2, 4], 13: [2, 7], 14: [4, 7], 15: [2, 4, 7], 16: [1, 2, 3, 4], 17: [0, 2, 3, 7], 18: [0, 1, 4, 7], 19: [0, 1], 20: [0, 2], 21: [1, 2], 22: [3, 4], 23: [0, 3, 4], 24: [0, 1, 2]}
{0: 0, 1: 1, 2: 2}


In [21]:
yields= []
final_params=[]
asymm = []
final_t50 = []
final_t85 = []
final_t95 = []
final_t99 = []
for i in range(len(optim.final_yields)):
    yields.append(optim.final_yields[i].item())
#     print(optim.final_solns[i].numpy())
    coupled_params=np.zeros(len(vec_rn.kon))
    if vec_rn.partial_opt:
        for j in range(len(vec_rn.kon)):
            if j in vec_rn.rx_cid.keys():
                all_rates=[]
                for rate in vec_rn.rx_cid[j]:
                    if rate in vec_rn.optim_rates:
                        all_rates.append(optim.final_solns[i][vec_rn.coup_map[rate]])
                    else:
                        if vec_rn.slow_rates is not None and rate in vec_rn.slow_rates:
                            all_rates.append(torch.mean(optim.final_solns[i][vec_rn.optim_rates])/vec_rn.slow_ratio)
                        else:
                            all_rates.append(vec_rn.kon[rate])
                coupled_params[j] = max(all_rates)
            else:
                if j in vec_rn.optim_rates:
                    coupled_params[j] = optim.final_solns[i][vec_rn.coup_map[j]]
                else:
                    if vec_rn.slow_rates is not None and j in vec_rn.slow_rates:       #Can be replaced later so that the RN figures out by iteself which are fast  interfaces and which are slow.
                        coupled_params[j] = torch.mean(optim.final_solns[i][vec_rn.optim_rates])/vec_rn.slow_ratio
                    else:
                        coupled_params[j] = vec_rn.kon[j]
    else:
        for j in range(len(vec_rn.kon)):
            if j in vec_rn.rx_cid.keys():
                #new_kon[i] = 1.0
                coupled_params[j] = max(optim.final_solns[i].numpy()[vec_rn.coup_map[rate]] for rate in vec_rn.rx_cid[j])
                # print("Max rate for reaction %s chosen as %.3f" %(i,self.coupled_kon[i]))
            else:
                coupled_params[j] = optim.final_solns[i].numpy()[vec_rn.coup_map[j]]
    final_params.append(coupled_params)
    
    if type(optim.final_t50[i])==int:
        final_t50.append(1) 
    else:
        final_t50.append(optim.final_t50[i].item()) 
    if type(optim.final_t85[i])==int:
        final_t85.append(1) 
    else:
        final_t85.append(optim.final_t85[i].item()) 
    if type(optim.final_t95[i])==int:
        final_t95.append(1)
    else:
        final_t95.append(optim.final_t95[i].item())

sort_indx=np.argsort(np.array(yields))
sorted_yields=np.array(yields)#[sort_indx]
sorted_params = np.array(final_params)#[sort_indx]

sorted_t50 = np.array(final_t50)#[sort_indx]
sorted_t85 = np.array(final_t85)#[sort_indx]
sorted_t95 = np.array(final_t95)#[sort_indx]

p0 = sorted_params[0]
var_params = []
for i in range(len(sorted_params)):
    var_params.append(calc_var(p0,sorted_params[i]))
    
    final_val,final_lb = calc_asymm(sorted_params[i])
    asymm.append(final_val)
    
arg_indx = np.argsort(np.array(var_params))
sorted_var = np.array(var_params)[arg_indx]

print(sorted_var[0])
print(sorted_var[-1])
print("Yield: ",sorted_yields[arg_indx[0]],"\nParams: ",sorted_params[arg_indx[0]])

print("Yield: ",sorted_yields[arg_indx[-1]],"\nParams: ",sorted_params[arg_indx[-1]])
print("Max Yield: ",sorted_yields[-1],"\nParams: ",list(sorted_params[-1]))

0.0
1.639729344757112
Yield:  0.8993929203648626 
Params:  [3.  3.  3.  0.1 0.1 3.  3.  0.1 3.  3.  0.1 3.  3.  3.  0.1 3.  3.  3.
 3.  3.  3.  3.  0.1 3.  3. ]
Yield:  0.9704421326998083 
Params:  [11.63576887 12.04529028 12.18627373  0.1         0.1        11.63576887
 11.63576887  0.1        12.04529028 12.04529028  0.1        12.04529028
 12.18627373 12.18627373  0.1        12.18627373 12.18627373 12.18627373
 12.04529028 12.04529028 12.18627373 12.18627373  0.1        11.63576887
 12.18627373]
Max Yield:  0.9704421326998083 
Params:  [11.635768867439248, 12.045290277260603, 12.186273732104679, 0.1, 0.1, 11.635768867439248, 11.635768867439248, 0.1, 12.045290277260603, 12.045290277260603, 0.1, 12.045290277260603, 12.186273732104679, 12.186273732104679, 0.1, 12.186273732104679, 12.186273732104679, 12.186273732104679, 12.045290277260603, 12.045290277260603, 12.186273732104679, 12.186273732104679, 0.1, 11.635768867439248, 12.186273732104679]


In [22]:
#Writing all solutions to a file

klabels=['k'+str(i) for i in range(len(vec_rn.kon))]
header = '#Yield\t' + "\t".join(klabels) + "\tt50\tt85\tt95\n"


with open("Solutions_Tetramer_Coupled_Irreversible",'w') as fl:
    fl.write(header)
    for i in range(len(sorted_yields)):
        fl.write("%f" %(sorted_yields[i]))
        
        for j in range((sorted_params[i].shape[0])):
            
            fl.write("\t%f" %(sorted_params[i][j]))
        fl.write("\t%f\t%f\t%f\n" %(sorted_t50[i],sorted_t85[i],sorted_t95[i]))
                 

In [ ]:
from matplotlib import pyplot as plt
n_features = len(asymm[0])
fig,ax = plt.subplots(int(n_features/2),2,figsize=(16,16))
# %matplotlib notebook

asymm = np.reshape(np.array(asymm),(len(sorted_yields),len(asymm[0])))
sorted_yields.reshape((sorted_yields.shape[0],1))

mask = (sorted_yields < 1.0) & (sorted_yields >0.5)

row=0
col=0
counter=0
for i in range(n_features):
    ax[row,col].plot(sorted_yields[mask],asymm[mask,i],marker='x',linestyle='',label=final_lb[i])
    ax[row,col].legend()
    ax[row,col].set_xlabel("Yield")
    ax[row,col].set_ylabel("Rate Values")
    
    counter+=1
    row = row+(col%2)
    col = counter%2
    
    

In [ ]:
from matplotlib import pyplot as plt
n_features = len(asymm[0])
fig,ax = plt.subplots(int(n_features/2),2,figsize=(16,16))
# %matplotlib notebook

asymm = np.reshape(np.array(asymm),(len(sorted_yields),len(asymm[0])))
sorted_yields.reshape((sorted_yields.shape[0],1))

mask = (sorted_yields < 0.9999) & (sorted_yields >0.90)

row=0
col=0
counter=0
for i in range(n_features):
    ax[row,col].plot(sorted_t85[mask],asymm[mask,i],marker='x',linestyle='',label='t85')
    ax[row,col].plot(sorted_t95[mask],asymm[mask,i],marker='x',linestyle='',label='t95')
    ax[row,col].plot(sorted_t50[mask],asymm[mask,i],marker='x',linestyle='',label='t50')
    ax[row,col].legend()
    ax[row,col].set_xlabel("Yield")
    ax[row,col].set_ylabel(final_lb[i])
    
    ax[row,col].set_xscale("log")
    
    
    counter+=1
    row = row+(col%2)
    col = counter%2

In [ ]:
#Let's see what some clustering reveals
#Is there a trend with high yield

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

def cluster_params(params,final_y,n_clust):
#     feat_mat = np.concatenate((params,final_y),axis=1)
    feat_mat = params
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(feat_mat)
    
    kmeans = KMeans(n_clusters = n_clust, random_state=0).fit(X_scaled)
    clus_cen = kmeans.cluster_centers_ #Obtain centroids for all the clusters
    transform_mat = kmeans.transform(feat_mat) #This method calculates the distance of each point from each cluster
    labels = kmeans.labels_  #Labels which frame belongs to which cluster
    
    return(labels,clus_cen,transform_mat)

In [ ]:
#Clustering
print(asymm.shape,sorted_yields.shape)
n_clust=8

var_feat_mat = asymm[mask][:,:]

labels,clus_cen,transform_mat = cluster_params(var_feat_mat,sorted_yields.reshape((sorted_yields.shape[0],1)),n_clust)


clr_input = ['skyblue','orange','green','red','purple','gold','brown','olive','crimson','peru','lightgreen','turquoise','cyan']

print(transform_mat.shape)


In [ ]:
feat_rates =sorted_params[mask]
sel_t50 = sorted_t50[mask]
sel_t85 = sorted_t85[mask]
sel_t95 = sorted_t95[mask]

asymm_new = asymm[mask]

#Create a bianry matrix to know which elements are form a cluster
mask_01=(np.array(labels)==0)
cluster_mask = mask_01.reshape((len(labels),1))
for i in range(1,n_clust):
    n_arr = i*np.ones((len(labels),1))-np.array(labels).reshape(((len(labels),1)))
    mask_01=(n_arr==0)
    print(n_arr.shape)
    cluster_mask=np.hstack((cluster_mask,mask_01))

# cluster_mask = cluster_mask.astype(float)

fig,ax = plt.subplots(int(n_features/2),2,figsize=(16,16))
ax_hd = []

clust_min_solutions={}
for i in range(n_clust):
    
    clus_1_dist = transform_mat[cluster_mask[:,i],i]
    clus_1_par = feat_rates[cluster_mask[:,i],:]
    clus_1_t50 = sel_t50[cluster_mask[:,i]]
    clus_1_t85 = sel_t85[cluster_mask[:,i]]
    clus_1_t95 = sel_t95[cluster_mask[:,i]]
    clus_1_asymm = asymm_new[cluster_mask[:,i]]
    indx_sort = np.argsort(clus_1_dist)
    
    print("Cluster: ",i)
    print("Cluster Centroid: ",clus_cen[i])
    sorted_dist = clus_1_dist[indx_sort]
    sorted_par = clus_1_par[indx_sort]
    clus_sorted_t50 = clus_1_t50[indx_sort]
    clus_sorted_t85 = clus_1_t85[indx_sort]
    clus_sorted_t95 = clus_1_t95[indx_sort]
    clus_sorted_asymm = clus_1_asymm[indx_sort]
    print("Max distance: ",sorted_dist[-1],"Params: ",sorted_par[-1])
    print("Min distance: ",sorted_dist[0],"Params: ",sorted_par[0])
    clust_min_solutions[i]=sorted_par[0]
    
    
    #Plotting
    row=0
    col=0
    counter=0
    for j in range(n_features):
        
#         h1=ax[row,col].scatter(clus_sorted_t85[-1],clus_sorted_asymm[-1,j],s=100,alpha=0.6,marker='o',color=clr_input[i], label='85%')
        h1 = ax[row,col].scatter(clus_sorted_t85[0:6],clus_sorted_asymm[0:6,j],s=200,alpha=0.6,marker='o',edgecolor=clr_input[i], facecolor='none', label='85%')
        
        ax[row,col].set_xlabel("Time taken",fontdict={'fontsize':'xx-large'},labelpad=1.0)
        ax[row,col].set_ylabel(final_lb[j],fontdict={'fontsize':'xx-large'},labelpad=2.0)

        ax[row,col].set_xscale("log")
        ax[row,col].tick_params(labelsize='xx-large')
        
        if counter==0:
            ax_hd.append(h1)
        counter+=1
        row = row+(col%2)
        col = counter%2
fig.legend(ax_hd,['C0','C1','C2','C3','C4','C5','C6','C7'],fontsize='x-large')
fig.tight_layout()

In [ ]:
uid_dict = {}
uid_reactants = {}
sys.path.append("../")
import numpy as np
from reaction_network import gtostr
for n in rn.network.nodes():
    #print(n)
    #print(rn.network.nodes()[n])
    for k,v in rn.network[n].items():
        uid = v['uid']
        r1 = set(gtostr(rn.network.nodes[n]['struct']))
        p = set(gtostr(rn.network.nodes[k]['struct']))
        r2 = p-r1
        reactants = (r1,r2)
        uid_val = {'uid':uid,'reactants':reactants,'kon':v['k_on'],'score':v['rxn_score'],'koff':v['k_off']}
        uid_reactants[uid]=reactants
        if uid not in uid_dict.keys():
            uid_dict[uid] = uid_val
    print(gtostr(rn.network.nodes[n]['struct']))
    #for r_set in rn.get_reactant_sets(n):
    #    print(tuple(r_set))
    #print(rn.network[n]['struct'])
ind_sort = np.argsort(vec_rn.kon.detach().numpy())
for i in ind_sort:
    print(vec_rn.kon[i])
    print(uid_dict[i])

In [ ]:
for cl_id,rids in vec_rn.rxn_class.items():
    
    print("------------------------------")
    print("------------------------------")
#     print("------    {:s}    -------".format(lab))
    print("------------------------------")
    print("-------------------------------")
    print("%-12s\t%-4s\t%-4s\t%-4s\t%-4s\t%-4s\t%-4s\t%-4s\t%-4s\t%-4s\n" %('Reaction','uid','C0','C1','C2','C3','C4','C5','C6','C7'))
    for r_id in rids:
        r1 = "".join(list(uid_reactants[r_id][0]))
        r2 = "".join(list(uid_reactants[r_id][1]))
        print("{:^4s} + {:^4s}".format(r1,r2),end='\t')
        print(r_id,end='\t')
        print(sorted_params[-1][r_id])
        print("")
    

In [ ]:
uid_dict = {}
sys.path.append("../")
import numpy as np
from reaction_network import gtostr
for n in rn.network.nodes():
    #print(n)
    #print(rn.network.nodes()[n])
    for k,v in rn.network[n].items():
        uid = v['uid']
        r1 = set(gtostr(rn.network.nodes[n]['struct']))
        p = set(gtostr(rn.network.nodes[k]['struct']))
        r2 = p-r1
        reactants = (r1,r2)
        uid_val = {'reactants':reactants,'kon':v['k_on'],'score':v['rxn_score'],'koff':v['k_off']}
        if uid not in uid_dict.keys():
            uid_dict[uid] = uid_val
    print(gtostr(rn.network.nodes[n]['struct']))
    #for r_set in rn.get_reactant_sets(n):
    #    print(tuple(r_set))
    #print(rn.network[n]['struct'])
ind_sort = np.argsort(vec_rn.kon.detach().numpy())
for i in ind_sort:
    print(vec_rn.kon[i])
    print(uid_dict[i])

In [ ]:
uid_dict = {}
sys.path.append("../")
import numpy as np
from reaction_network import gtostr
from torch import DoubleTensor as Tensor

def get_max_edge(n):
    """
    Calculates the max rate (k_on) for a given node
    To find out the maximum flow path to the final complex starting from the current node.
    
    Can also calculate the total rate of consumption of a node by summing up all rates. 
    Can tell which component is used quickly.
    """
    try:
        edges = rn.network.out_edges(n)
        #Loop over all edges
        #Get attributes
        if len(edges)==0:
            return(False)
        kon_max = -1
        next_node = -1
        
        kon_sum = 0
        for edge in edges:
            data = rn.network.get_edge_data(edge[0],edge[1])
            #print(data)
            #Get uid
            uid = data['uid']
            #Get updated kon
            temp_kon = vec_rn.kon[uid]
            kon_sum+=temp_kon
            
#             #Calculate k_off also
#             std_c = Tensor([1.])
#             l_kon = torch.log(temp_kon)
#             l_koff = (vec_rn.rxn_score_vec[uid] * 1. / (self._R * self._T)) + l_kon + torch.log(std_c)
            if temp_kon > kon_max:
                kon_max = temp_kon
                next_node=edge[1]
        return(kon_max,next_node,kon_sum)
    except Exception as err:
        raise(err)

pathway = []
kon_sumarray = []
total_con_rate = {}
for n in rn.network.nodes():
    
    n_str = gtostr(rn.network.nodes[n]['struct']) 
    
    paths = [n_str]
    kon_sum = 0
    temp_node = n
    max_edge = True
    consumption_rate = 0
    if n < len(rn.network.nodes()):#num_monomers:
#         print("Current node: ")
#         print(n_str)
        while max_edge:
            max_edge = get_max_edge(temp_node)
            if max_edge:
                total_con_rate[gtostr(rn.network.nodes[temp_node]['struct'])] = max_edge[2]
                temp_node = max_edge[1]
                kon_sum += max_edge[0].item()
                
#                 print("Next node: ")
#                 print(temp_node)

                paths.append(gtostr(rn.network.nodes[temp_node]['struct']))
            else:
                break
        pathway.append(paths)
        kon_sumarray.append(kon_sum)
        paths=[]

print(pathway)
print(kon_sumarray)
#print(total_con_rate)

In [ ]:
for k,v in sorted(total_con_rate.items(),key=lambda x : x[1]):
    print(k," : ", v.item())

Let's first visualize some of the data.

**Without any optimization**


In [ ]:
nodes_list = ['A','B','S','M','AB','BMS','ABS','AMS','ABMS','AM','AS']
#nodes_list = ['A','B','ABMS']
optim.plot_observable(0,nodes_list)


**After 750 optimization iterations**


In [ ]:
optim.plot_observable(-1,nodes_list)


In [ ]:
optim.plot_yield()

It seems like we've found a stable solution that produces greater yield than equilibrium. This should be thermodynamically
impossible. Let's try to find an explanation. We'll run simulations using the learned optimal parameters at a few different
timescales.

In [ ]:
from matplotlib import pyplot as plt
fig, ax = plt.subplots(1, 3)
optim_rn = optim.rn
for i, runtime in enumerate([1, 8, 64]):
    optim_rn.reset()
    sim = VecSim(optim_rn, runtime, device='cpu')
    y = sim.simulate()
    sim.plot_observable(nodes_list,ax=ax[i],)
    ax[i].set_title("runtime: " + str(runtime) + " seconds")
fig.set_size_inches(18, 6)
node_map = {}
for node in rn.network.nodes():
    node_map[gtostr(rn.network.nodes[node]['struct'])] = node

print(node_map)
plt.show()

In [ ]:
node_map = {}
for node in rn.network.nodes():
    node_map[gtostr(rn.network.nodes[node]['struct'])] = node

print(node_map)
def get_max_edge(n):
    """
    Calculates the max rate (k_on) for a given node
    To find out the maximum flow path to the final complex starting from the current node.
    
    Can also calculate the total rate of consumption of a node by summing up all rates. 
    Can tell which component is used quickly.
    """
    try:
        edges = rn.network.out_edges(n)
        #Loop over all edges
        #Get attributes
        kon_max = -1
        next_node = -1

        kon_sum = 0
        total_flux_outedges = 0
        total_flux_inedges = 0
        if len(edges)==0:
            return(False)
            
        for edge in edges:
            data = rn.network.get_edge_data(edge[0],edge[1])
            #print(data)
            #Get uid
            uid = data['uid']

            #Get updated kon
            temp_kon = vec_rn.kon[uid]
            kon_sum+=temp_kon
            
            if temp_kon > kon_max:
                kon_max = temp_kon
                next_node=edge[1]
             
        return(kon_max,next_node,kon_sum)
    except Exception as err:
        raise(err)

        
def get_node_flux(n):
    total_flux_outedges = 0
    total_flux_inedges = 0
    #Go over all the out edges
    edges_out = rn.network.out_edges(n)
    if len(edges_out)>0:

        for edge in edges_out:
            data = rn.network.get_edge_data(edge[0],edge[1])
            #print(data)
            #Get uid
            uid = data['uid']

            #Get updated kon
            temp_kon = vec_rn.kon[uid]

            #Calculate k_off also
            std_c = Tensor([1.])
            l_kon = torch.log(temp_kon)
            l_koff = (vec_rn.rxn_score_vec[uid] * 1. / (vec_rn._R * vec_rn._T)) + l_kon + torch.log(std_c)
            koff = torch.exp(l_koff)

            #Getting conc. of reactants and products
            #Get product
            prod = gtostr(rn.network.nodes[edge[1]]['struct']) 
            #Get other reactant
            react = "".join(sorted(list(set(prod) - set(gtostr(rn.network.nodes[edge[0]]['struct']) ))))

            #Net flux from this edge = Generation - consumption
            edge_flux = koff*vec_rn.copies_vec[edge[1]] - temp_kon*(vec_rn.copies_vec[edge[0]])*(vec_rn.copies_vec[node_map[react]])
            #edge_flux = koff*vec_rn.copies_vec[edge[1]] 

            print("Reaction: ", gtostr(rn.network.nodes[edge[0]]['struct']), "+",react," -> ",prod)
            print("Net flux: ",edge_flux)
            print("kon : ",temp_kon)
            print("koff: ",koff)
            print("Reaction data OUTWARD: ")
            print(data)

            total_flux_outedges+=edge_flux
    
    #Now go over all the in edges
    edges_in = rn.network.in_edges(n)
    react_list = []
    if len(edges_in) > 0:
        for edge in edges_in:
            if edge[0] in react_list:
                continue
            data = rn.network.get_edge_data(edge[0],edge[1])
            uid = data['uid']


            #Get generation rates; which would be kon
            temp_kon = vec_rn.kon[uid]

            #Get consumption rates; which is k_off
            std_c = Tensor([1.])
            l_kon = torch.log(temp_kon)
            l_koff = (vec_rn.rxn_score_vec[uid] * 1. / (vec_rn._R * vec_rn._T)) + l_kon + torch.log(std_c)
            koff = torch.exp(l_koff)

            #Get conc. of reactants and products
            prod = gtostr(rn.network.nodes[edge[1]]['struct'])
            #Get other reactant
            react = "".join(sorted(list(set(prod) - set(gtostr(rn.network.nodes[edge[0]]['struct']) ))))
            react_list.append(node_map[react])
            #Net flux from this edge = Generation - consumption
            edge_flux_in = temp_kon*(vec_rn.copies_vec[edge[0]])*(vec_rn.copies_vec[node_map[react]])- koff*vec_rn.copies_vec[edge[1]]
            #edge_flux_in = koff*vec_rn.copies_vec[edge[1]]
            


            print("Reaction: ", prod ," -> ",gtostr(rn.network.nodes[edge[0]]['struct']), "+",react)
            print("Net flux: ",edge_flux_in)
            print("kon : ",temp_kon)
            print("koff: ",koff)
            print("Raction data INWARD: ")
            print(data)

            total_flux_inedges+=edge_flux_in
    net_node_flux = total_flux_outedges + total_flux_inedges
    
    return(net_node_flux)
    
pathway = []
kon_sumarray = []
total_con_rate = {}
net_flux = {}
for n in rn.network.nodes():
    
    n_str = gtostr(rn.network.nodes[n]['struct']) 
    
    paths = [n_str]
    kon_sum = 0
    temp_node = n
    max_edge = True
    consumption_rate = 0
    if n < len(rn.network.nodes()):#num_monomers:
#         print("Current node: ")
#         print(n_str)
        while max_edge:
            max_edge = get_max_edge(temp_node)
            if max_edge:
                total_con_rate[gtostr(rn.network.nodes[temp_node]['struct'])] = max_edge[2]
                
                temp_node = max_edge[1]
                kon_sum += max_edge[0].item()
                
                
#                 print("Next node: ")
#                 print(temp_node)

                paths.append(gtostr(rn.network.nodes[temp_node]['struct']))
            else:
                break
        pathway.append(paths)
        kon_sumarray.append(kon_sum)
        paths=[]
    print("-------------------------------------------------------------------------------")
    print("-------------------------------------------------------------------------------")
    print("|                                                                             |")
    node_flux = get_node_flux(n)
    net_flux[gtostr(rn.network.nodes[n]['struct'])] = node_flux
    print("|                                                                             |")
    print("-------------------------------------------------------------------------------")
    print("-------------------------------------------------------------------------------")

print(pathway)
print(kon_sumarray)

#print(total_con_rate)

In [ ]:
for k,v in sorted(net_flux.items(),key=lambda x : x[1]):
    print(k," : ", v)

print(vec_rn.copies_vec)
print(vec_rn.kon)

In [ ]:
print(solution)
poly_system = EquilibriumSolver(rn)
solution = poly_system.solve(init_val=vec_rn.copies_vec.detach().numpy().tolist())
#solution = poly_system.solve(verifyBool = False)
if solution == None:
    print("No Equilibrium solution")
else:
    print(solution)
    print("Equilibrium expected yield: ", 100 * solution[-1] / min(vec_rn.initial_copies[:vec_rn.num_monomers]), '%')
print(vec_rn.kon)

Clearly, the equilibrium reached by the system still matches the equilibrium solution. We have however found a set of parameters that can increase available complete AP2 at some point before equilibrium to levels significantly higher than at equilibrium. We don't observe any trapping, but have uncovered an interesting effect. 

Now we'll move on to looking at ARP23. This is 7 subunits, which drastically increases the number of possible reactions. Expect longer runtimes. 